In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
train_joke_df = pd.read_csv(r'..\data\recsys-in-practice\train_joke_df.csv')

joke_df = pd.read_csv(r'..\joke_doc2vec_vectors.csv')

In [4]:
train_joke_df = train_joke_df.merge(joke_df, left_on='JID', right_index=True)
train_joke_df

,UID,JID,Rating,joke_feature_1,joke_feature_2,joke_feature_3,joke_feature_4,joke_feature_5,joke_feature_6,joke_feature_7,...,joke_feature_91,joke_feature_92,joke_feature_93,joke_feature_94,joke_feature_95,joke_feature_96,joke_feature_97,joke_feature_98,joke_feature_99,joke_feature_100
0,18029,6,-1.26,-0.036663,-0.067076,0.023498,-0.033260,0.055063,-0.029962,0.007167,...,-0.057065,-0.026336,-0.019614,-0.023196,0.053547,0.026594,0.018127,-0.023290,-0.042454,-0.008761
17,5621,6,3.16,-0.036663,-0.067076,0.023498,-0.033260,0.055063,-0.029962,0.007167,...,-0.057065,-0.026336,-0.019614,-0.023196,0.053547,0.026594,0.018127,-0.023290,-0.042454,-0.008761
101,17251,6,-3.74,-0.036663,-0.067076,0.023498,-0.033260,0.055063,-0.029962,0.007167,...,-0.057065,-0.026336,-0.019614,-0.023196,0.053547,0.026594,0.018127,-0.023290,-0.042454,-0.008761
142,2786,6,6.36,-0.036663,-0.067076,0.023498,-0.033260,0.055063,-0.029962,0.007167,...,-0.057065,-0.026336,-0.019614,-0.023196,0.053547,0.026594,0.018127,-0.023290,-0.042454,-0.008761
273,14079,6,2.09,-0.036663,-0.067076,0.023498,-0.033260,0.055063,-0.029962,0.007167,...,-0.057065,-0.026336,-0.019614,-0.023196,0.053547,0.026594,0.018127,-0.023290,-0.042454,-0.008761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447099,22356,74,1.89,0.004980,0.003973,-0.010549,-0.004713,0.029614,0.002287,-0.001566,...,-0.014091,-0.017337,-0.000019,-0.010038,0.009264,0.035359,0.022170,-0.013675,0.009155,-0.004111
1447290,4234,74,7.04,0.004980,0.003973,-0.010549,-0.004713,0.029614,0.002287,-0.001566,...,-0.014091,-0.017337,-0.000019,-0.010038,0.009264,0.035359,0.022170,-0.013675,0.009155,-0.004111
1447538,16115,74,0.34,0.004980,0.003973,-0.010549,-0.004713,0.029614,0.002287,-0.001566,...,-0.014091,-0.017337,-0.000019,-0.010038,0.009264,0.035359,0.022170,-0.013675,0.009155,-0.004111
1447727,11579,74,6.17,0.004980,0.003973,-0.010549,-0.004713,0.029614,0.002287,-0.001566,...,-0.014091,-0.017337,-0.000019,-0.010038,0.009264,0.035359,0.022170,-0.013675,0.009155,-0.004111


In [5]:
train_df, valid_df = train_test_split(train_joke_df, test_size=0.001, random_state=42)

In [6]:
# сделаем сортировку и перепишем index
train_df = train_df.sort_values(by=['UID', 'JID'])
train_df = train_df.reset_index(drop=True)

valid_df = valid_df.sort_values(by=['UID', 'JID'])
valid_df = valid_df.reset_index(drop=True)

In [7]:
from catboost import CatBoostRanker, Pool, MetricVisualizer, CatBoostRegressor
from copy import deepcopy

In [8]:
cat_features = ['UID', 'JID']

In [9]:
train_pool = Pool(train_df.drop(columns='Rating'), label=train_df['Rating'], group_id=train_df['UID'],cat_features=cat_features)
valid_pool = Pool(valid_df.drop(columns='Rating'), label=valid_df['Rating'], group_id=valid_df['UID'],cat_features=cat_features)

In [10]:
default_parameters = {
    'iterations': 2000,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE'
}


In [11]:
model = CatBoostRegressor(**default_parameters)
model.fit(train_pool, eval_set=valid_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.104492
0:	learn: 5.1147751	test: 4.9375150	best: 4.9375150 (0)	total: 575ms	remaining: 19m 9s
1:	learn: 5.0154372	test: 4.8301719	best: 4.8301719 (1)	total: 942ms	remaining: 15m 41s
2:	learn: 4.9350852	test: 4.7436517	best: 4.7436517 (2)	total: 1.22s	remaining: 13m 28s
3:	learn: 4.8678156	test: 4.6709476	best: 4.6709476 (3)	total: 1.48s	remaining: 12m 16s
4:	learn: 4.8103127	test: 4.6124720	best: 4.6124720 (4)	total: 1.73s	remaining: 11m 30s
5:	learn: 4.7631606	test: 4.5634413	best: 4.5634413 (5)	total: 2.07s	remaining: 11m 28s
6:	learn: 4.7227985	test: 4.5227882	best: 4.5227882 (6)	total: 2.34s	remaining: 11m 5s
7:	learn: 4.6901685	test: 4.4894800	best: 4.4894800 (7)	total: 2.54s	remaining: 10m 32s
8:	learn: 4.6623588	test: 4.4601710	best: 4.4601710 (8)	total: 2.81s	remaining: 10m 22s
9:	learn: 4.6400327	test: 4.4363535	best: 4.4363535 (9)	total: 3.25s	remaining: 10m 46s
10:	learn: 4.6211545	test: 4.4183295	best: 4.4183295 (10)	total: 3.45s	remaining: 10m 24s
11

92:	learn: 4.5009945	test: 4.3120457	best: 4.3101865 (54)	total: 35.6s	remaining: 12m 10s
93:	learn: 4.5009174	test: 4.3119175	best: 4.3101865 (54)	total: 36.1s	remaining: 12m 12s
94:	learn: 4.5008600	test: 4.3116563	best: 4.3101865 (54)	total: 36.7s	remaining: 12m 15s
95:	learn: 4.5007875	test: 4.3114786	best: 4.3101865 (54)	total: 37.1s	remaining: 12m 16s
96:	learn: 4.5006162	test: 4.3113258	best: 4.3101865 (54)	total: 37.6s	remaining: 12m 17s
97:	learn: 4.5005520	test: 4.3114597	best: 4.3101865 (54)	total: 38.1s	remaining: 12m 20s
98:	learn: 4.5004035	test: 4.3112745	best: 4.3101865 (54)	total: 38.5s	remaining: 12m 20s
99:	learn: 4.5002371	test: 4.3111935	best: 4.3101865 (54)	total: 39.1s	remaining: 12m 22s
100:	learn: 4.5001127	test: 4.3111926	best: 4.3101865 (54)	total: 39.4s	remaining: 12m 20s
101:	learn: 4.5000804	test: 4.3111510	best: 4.3101865 (54)	total: 39.6s	remaining: 12m 16s
102:	learn: 4.4999975	test: 4.3111378	best: 4.3101865 (54)	total: 40s	remaining: 12m 17s
103:	lear

182:	learn: 4.4924811	test: 4.3039463	best: 4.3039463 (182)	total: 1m 18s	remaining: 12m 56s
183:	learn: 4.4923922	test: 4.3036389	best: 4.3036389 (183)	total: 1m 18s	remaining: 12m 57s
184:	learn: 4.4923391	test: 4.3034612	best: 4.3034612 (184)	total: 1m 19s	remaining: 12m 55s
185:	learn: 4.4922739	test: 4.3034774	best: 4.3034612 (184)	total: 1m 19s	remaining: 12m 53s
186:	learn: 4.4921495	test: 4.3033246	best: 4.3033246 (186)	total: 1m 19s	remaining: 12m 54s
187:	learn: 4.4920638	test: 4.3034289	best: 4.3033246 (186)	total: 1m 20s	remaining: 12m 55s
188:	learn: 4.4920186	test: 4.3035724	best: 4.3033246 (186)	total: 1m 20s	remaining: 12m 55s
189:	learn: 4.4919828	test: 4.3035153	best: 4.3033246 (186)	total: 1m 21s	remaining: 12m 57s
190:	learn: 4.4918902	test: 4.3036971	best: 4.3033246 (186)	total: 1m 22s	remaining: 12m 58s
191:	learn: 4.4916907	test: 4.3031612	best: 4.3031612 (191)	total: 1m 22s	remaining: 13m
192:	learn: 4.4916511	test: 4.3032161	best: 4.3031612 (191)	total: 1m 23s	

271:	learn: 4.4858428	test: 4.2999215	best: 4.2999215 (271)	total: 2m	remaining: 12m 46s
272:	learn: 4.4858047	test: 4.2996881	best: 4.2996881 (272)	total: 2m 1s	remaining: 12m 46s
273:	learn: 4.4856138	test: 4.2995562	best: 4.2995562 (273)	total: 2m 1s	remaining: 12m 46s
274:	learn: 4.4855491	test: 4.2995209	best: 4.2995209 (274)	total: 2m 2s	remaining: 12m 45s
275:	learn: 4.4854181	test: 4.2992884	best: 4.2992884 (275)	total: 2m 2s	remaining: 12m 44s
276:	learn: 4.4853545	test: 4.2994764	best: 4.2992884 (275)	total: 2m 2s	remaining: 12m 44s
277:	learn: 4.4853233	test: 4.2992986	best: 4.2992884 (275)	total: 2m 3s	remaining: 12m 42s
278:	learn: 4.4852833	test: 4.2992407	best: 4.2992407 (278)	total: 2m 3s	remaining: 12m 42s
279:	learn: 4.4852418	test: 4.2992975	best: 4.2992407 (278)	total: 2m 3s	remaining: 12m 41s
280:	learn: 4.4852147	test: 4.2993578	best: 4.2992407 (278)	total: 2m 4s	remaining: 12m 41s
281:	learn: 4.4852007	test: 4.2993522	best: 4.2992407 (278)	total: 2m 5s	remaining:

...

1599:	learn: 4.4415541	test: 4.2552401	best: 4.2550954 (1597)	total: 12m 42s	remaining: 3m 10s
1600:	learn: 4.4415361	test: 4.2552380	best: 4.2550954 (1597)	total: 12m 43s	remaining: 3m 10s
1601:	learn: 4.4415045	test: 4.2552609	best: 4.2550954 (1597)	total: 12m 43s	remaining: 3m 9s
1602:	learn: 4.4414967	test: 4.2553155	best: 4.2550954 (1597)	total: 12m 44s	remaining: 3m 9s
1603:	learn: 4.4414567	test: 4.2553186	best: 4.2550954 (1597)	total: 12m 45s	remaining: 3m 8s
1604:	learn: 4.4414333	test: 4.2551964	best: 4.2550954 (1597)	total: 12m 45s	remaining: 3m 8s
1605:	learn: 4.4414135	test: 4.2551073	best: 4.2550954 (1597)	total: 12m 46s	remaining: 3m 7s
1606:	learn: 4.4413719	test: 4.2552184	best: 4.2550954 (1597)	total: 12m 46s	remaining: 3m 7s
1607:	learn: 4.4413558	test: 4.2551662	best: 4.2550954 (1597)	total: 12m 47s	remaining: 3m 6s
1608:	learn: 4.4413413	test: 4.2551356	best: 4.2550954 (1597)	total: 12m 47s	remaining: 3m 6s
1609:	learn: 4.4412957	test: 4.2552000	best: 4.2550954 (15

1686:	learn: 4.4387958	test: 4.2522950	best: 4.2522950 (1686)	total: 13m 27s	remaining: 2m 29s
1687:	learn: 4.4387785	test: 4.2523029	best: 4.2522950 (1686)	total: 13m 27s	remaining: 2m 29s
1688:	learn: 4.4387338	test: 4.2521608	best: 4.2521608 (1688)	total: 13m 28s	remaining: 2m 28s
1689:	learn: 4.4386917	test: 4.2521418	best: 4.2521418 (1689)	total: 13m 28s	remaining: 2m 28s
1690:	learn: 4.4386710	test: 4.2522198	best: 4.2521418 (1689)	total: 13m 29s	remaining: 2m 27s
1691:	learn: 4.4386465	test: 4.2522125	best: 4.2521418 (1689)	total: 13m 29s	remaining: 2m 27s
1692:	learn: 4.4385450	test: 4.2522582	best: 4.2521418 (1689)	total: 13m 30s	remaining: 2m 26s
1693:	learn: 4.4385272	test: 4.2521706	best: 4.2521418 (1689)	total: 13m 30s	remaining: 2m 26s
1694:	learn: 4.4385081	test: 4.2522429	best: 4.2521418 (1689)	total: 13m 31s	remaining: 2m 25s
1695:	learn: 4.4384821	test: 4.2522319	best: 4.2521418 (1689)	total: 13m 31s	remaining: 2m 25s
1696:	learn: 4.4384654	test: 4.2523214	best: 4.252

1773:	learn: 4.4362010	test: 4.2481746	best: 4.2481746 (1773)	total: 14m 11s	remaining: 1m 48s
1774:	learn: 4.4361754	test: 4.2481805	best: 4.2481746 (1773)	total: 14m 11s	remaining: 1m 47s
1775:	learn: 4.4361478	test: 4.2480245	best: 4.2480245 (1775)	total: 14m 11s	remaining: 1m 47s
1776:	learn: 4.4360857	test: 4.2476543	best: 4.2476543 (1776)	total: 14m 12s	remaining: 1m 46s
1777:	learn: 4.4360767	test: 4.2476136	best: 4.2476136 (1777)	total: 14m 12s	remaining: 1m 46s
1778:	learn: 4.4360375	test: 4.2475083	best: 4.2475083 (1778)	total: 14m 13s	remaining: 1m 46s
1779:	learn: 4.4359662	test: 4.2469807	best: 4.2469807 (1779)	total: 14m 14s	remaining: 1m 45s
1780:	learn: 4.4359497	test: 4.2470083	best: 4.2469807 (1779)	total: 14m 14s	remaining: 1m 45s
1781:	learn: 4.4359394	test: 4.2470631	best: 4.2469807 (1779)	total: 14m 15s	remaining: 1m 44s
1782:	learn: 4.4358949	test: 4.2469815	best: 4.2469807 (1779)	total: 14m 15s	remaining: 1m 44s
1783:	learn: 4.4358800	test: 4.2468770	best: 4.246

1860:	learn: 4.4340570	test: 4.2452902	best: 4.2451746 (1826)	total: 14m 54s	remaining: 1m 6s
1861:	learn: 4.4340314	test: 4.2450995	best: 4.2450995 (1861)	total: 14m 55s	remaining: 1m 6s
1862:	learn: 4.4340104	test: 4.2451192	best: 4.2450995 (1861)	total: 14m 55s	remaining: 1m 5s
1863:	learn: 4.4339880	test: 4.2450592	best: 4.2450592 (1863)	total: 14m 56s	remaining: 1m 5s
1864:	learn: 4.4339511	test: 4.2450265	best: 4.2450265 (1864)	total: 14m 56s	remaining: 1m 4s
1865:	learn: 4.4339239	test: 4.2449341	best: 4.2449341 (1865)	total: 14m 57s	remaining: 1m 4s
1866:	learn: 4.4338851	test: 4.2447140	best: 4.2447140 (1866)	total: 14m 57s	remaining: 1m 3s
1867:	learn: 4.4338357	test: 4.2446576	best: 4.2446576 (1867)	total: 14m 58s	remaining: 1m 3s
1868:	learn: 4.4337742	test: 4.2445462	best: 4.2445462 (1868)	total: 14m 58s	remaining: 1m 2s
1869:	learn: 4.4337633	test: 4.2445199	best: 4.2445199 (1869)	total: 14m 59s	remaining: 1m 2s
1870:	learn: 4.4337499	test: 4.2445092	best: 4.2445092 (1870

1948:	learn: 4.4317018	test: 4.2427415	best: 4.2425014 (1946)	total: 15m 40s	remaining: 24.6s
1949:	learn: 4.4316787	test: 4.2427748	best: 4.2425014 (1946)	total: 15m 40s	remaining: 24.1s
1950:	learn: 4.4316196	test: 4.2424581	best: 4.2424581 (1950)	total: 15m 41s	remaining: 23.6s
1951:	learn: 4.4316004	test: 4.2424655	best: 4.2424581 (1950)	total: 15m 42s	remaining: 23.2s
1952:	learn: 4.4315610	test: 4.2423719	best: 4.2423719 (1952)	total: 15m 42s	remaining: 22.7s
1953:	learn: 4.4315533	test: 4.2423646	best: 4.2423646 (1953)	total: 15m 42s	remaining: 22.2s
1954:	learn: 4.4315281	test: 4.2424627	best: 4.2423646 (1953)	total: 15m 43s	remaining: 21.7s
1955:	learn: 4.4315117	test: 4.2424658	best: 4.2423646 (1953)	total: 15m 43s	remaining: 21.2s
1956:	learn: 4.4314702	test: 4.2422604	best: 4.2422604 (1956)	total: 15m 44s	remaining: 20.7s
1957:	learn: 4.4314467	test: 4.2421303	best: 4.2421303 (1957)	total: 15m 44s	remaining: 20.3s
1958:	learn: 4.4314144	test: 4.2421868	best: 4.2421303 (1957

In [17]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))

predict = model.predict(train_pool)
print(mean_squared_error(train_df['Rating'].values, predict, squared=False))


4.241985282938053
4.198584177741335


In [18]:
model_name = "catboost_doc2vec"

In [19]:
model.save_model(model_name)

from_file = CatBoostRegressor()

from_file.load_model(model_name)


In [20]:
predict = from_file.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))

predict = from_file.predict(train_pool)
print(mean_squared_error(train_df['Rating'].values, predict, squared=False))


4.241985282938053
4.198584177741335


In [16]:
assert False

AssertionError: 

In [ ]:
import gc

In [ ]:
gc.collect()

In [ ]:
del train_pool

In [22]:
test_joke_df_nofactrating = pd.read_csv(r'..\data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)

test_joke_df_nofactrating = test_joke_df_nofactrating.merge(joke_df, how='left', left_on='JID', right_index=True)
test_joke_df_nofactrating
test_joke_df_nofactrating = test_joke_df_nofactrating.sort_values(by=['UID', 'JID'])
test_pool = Pool(test_joke_df_nofactrating, group_id=test_joke_df_nofactrating['UID'], cat_features=cat_features)

In [23]:
predict = model.predict(test_pool)

test_joke_df_nofactrating['Rating'] = predict

display(test_joke_df_nofactrating['Rating'].to_frame().head(5))
test_joke_df_nofactrating['Rating'].to_frame().to_csv('catboost_doc2vec.csv')

,Rating
InteractionID,
219361,-0.580244
83962,-1.856137
64748,-2.264332
83931,0.198785
170573,-1.693747


In [24]:
test_joke_df_nofactrating

,UID,JID,joke_feature_1,joke_feature_2,joke_feature_3,joke_feature_4,joke_feature_5,joke_feature_6,joke_feature_7,joke_feature_8,...,joke_feature_92,joke_feature_93,joke_feature_94,joke_feature_95,joke_feature_96,joke_feature_97,joke_feature_98,joke_feature_99,joke_feature_100,Rating
InteractionID,,,,,,,,,,,,,,,,,,,,,
219361,1,11,0.026352,-0.001528,-0.013753,0.009290,-0.025492,-0.011763,-0.015875,-0.034059,...,0.025123,0.020059,0.027965,-0.004149,-0.042765,-0.015268,0.006480,0.004333,0.014611,-0.580244
83962,1,19,0.042389,0.000730,-0.035406,0.036010,-0.052547,-0.028098,0.004075,-0.077485,...,0.004668,0.021024,0.005693,0.000952,-0.075436,-0.009181,0.012338,0.019292,0.019831,-1.856137
64748,1,25,-0.000098,-0.008685,-0.010810,0.011853,-0.010585,0.003910,0.001658,-0.013394,...,0.005005,0.000969,-0.010692,-0.002849,0.000785,-0.012602,-0.000385,0.001842,0.000569,-2.264332
83931,1,31,0.066213,0.069451,-0.073179,0.078512,-0.079443,0.025186,-0.020673,-0.103958,...,0.064038,0.046252,0.030843,-0.059683,-0.103271,-0.021781,0.058302,0.062486,0.042500,0.198785
170573,1,34,0.016923,0.004841,-0.007198,0.005487,-0.003737,0.004410,-0.004767,-0.007779,...,0.013120,0.003966,-0.001303,-0.002993,-0.018210,-0.000988,0.003183,0.004446,-0.001467,-1.693747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291554,24983,60,0.010755,0.005413,0.009041,-0.023843,0.018403,0.020030,-0.030876,0.013676,...,0.085763,-0.042025,0.075884,-0.064349,-0.025406,-0.065871,-0.024266,0.069343,0.036641,1.195054
21758,24983,61,-0.016029,-0.001883,0.007151,-0.003661,0.009370,0.005577,0.009364,0.022427,...,-0.001736,-0.015726,-0.000801,-0.001240,0.020270,0.005188,-0.008636,-0.003013,0.000872,4.185934
14611,24983,63,-0.036177,-0.059600,0.033925,-0.043557,0.084298,-0.044493,0.014303,0.059161,...,-0.062363,0.011567,-0.060439,0.053983,0.102408,0.039643,-0.058870,-0.065539,-0.039887,2.241097
